In [1]:
from pyspark.sql import SparkSession
from pyproj import Proj,transform, CRS
import numpy as np
import pandas as pd
import mysql.connector
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from tqdm import tqdm

In [2]:
# Spark 세션 시작
spark = SparkSession.builder \
    .appName("CSV to MariaDB") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 07:29:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# CSV 파일 읽기
df = spark.read.csv("hdfs://localhost:9000/data/구polygon.csv", header=True, inferSchema=True)

In [ ]:
df.printSchema()

In [ ]:
df.show()

In [6]:
# EPSG 코드 정의 (권장되는 최신 방식)
proj_UTMK = Proj(CRS.from_epsg(5181))
proj_WGS84 = Proj(CRS.from_epsg(4326))

In [ ]:
x1,y1 = 197800.7194, 459064.2454
x2,y2 = transform(proj_UTMK,proj_WGS84,x1,y1)
print(x2,y2)

In [8]:
# 좌표 변환 함수 정의
def utmk_to_wgs84(x, y):
    lon, lat = transform(proj_UTMK, proj_WGS84, x, y)
    return float(lon), float(lat)

def transform_polygon(geometry_str):
    def transform_coords(coords_str):
        transformed_coords = []
        for coord_pair in coords_str.split(", "):
            x, y = map(float, coord_pair.split())
            lon, lat = utmk_to_wgs84(x, y)
            transformed_coords.append(f"{lon} {lat}")
        return transformed_coords
    # "POLYGON"을 기준으로 좌표 추출
    if geometry_str.startswith("POLYGON"):
        coordinates_str = geometry_str.replace("POLYGON ((", "").replace("))", "")
        # 개별 좌표쌍을 변환
        transformed_coords = []
        for coord_pair in tqdm(coordinates_str.split(", ")):
            x, y = map(float, coord_pair.split())
            lon, lat = utmk_to_wgs84(x, y)
            transformed_coords.append(f"{lon} {lat}")
        # 변환된 좌표들을 다시 폴리곤 형식으로 복원
        transformed_polygon = "POLYGON ((" + ", ".join(transformed_coords) + "))"
        return transformed_polygon
    elif geometry_str.startswith("MULTIPOLYGON"):
        # MULTIPOLYGON 좌표 추출 및 변환
        multipolygons_str = geometry_str.replace("MULTIPOLYGON (((", "").replace(")))", "")
        individual_polygons = multipolygons_str.split(")), ((")  # 각 POLYGON을 구분
        transformed_polygons = []
        
        for polygon_str in individual_polygons:
            transformed_coords = transform_coords(polygon_str)
            transformed_polygon = "((" + ", ".join(transformed_coords) + "))"
            transformed_polygons.append(transformed_polygon)
        
        transformed_multipolygon = "MULTIPOLYGON (" + ", ".join(transformed_polygons) + ")"
        return transformed_multipolygon
    return geometry_str

# UDF를 통해 변환 함수 등록
utmk_to_wgs84_udf_lon = udf(lambda x, y: utmk_to_wgs84(x, y)[0], FloatType())
utmk_to_wgs84_udf_lat = udf(lambda x, y: utmk_to_wgs84(x, y)[1], FloatType())
transform_polygon_udf = udf(transform_polygon, StringType())

In [ ]:


# 새로운 x, y 컬럼 추가 (WGS84로 변환)
df = df.withColumn("x", utmk_to_wgs84_udf_lon(df["X좌표"], df["Y좌표"]))
df = df.withColumn("y", utmk_to_wgs84_udf_lat(df["X좌표"], df["Y좌표"]))

# geometry 컬럼 좌표 변환
df = df.withColumn("transformed_geometry", transform_polygon_udf(df["geometry"]))

# 결과 출력
df.show(truncate=False)

In [ ]:
# CSV 파일 읽기
seoul_df = spark.read.csv("hdfs://localhost:9000/data/서울polygon.csv", header=True, inferSchema=True)
seoul_df

In [ ]:
# 새로운 x, y 컬럼 추가 (WGS84로 변환)
seoul_df = seoul_df.withColumn("x", utmk_to_wgs84_udf_lon(seoul_df["X좌표"], seoul_df["Y좌표"]))
seoul_df = seoul_df.withColumn("y", utmk_to_wgs84_udf_lat(seoul_df["X좌표"], seoul_df["Y좌표"]))

# geometry 컬럼 좌표 변환
seoul_df = seoul_df.withColumn("transformed_geometry", transform_polygon_udf(seoul_df["geometry"]))

# 결과 출력
seoul_df.show(truncate=False)

In [ ]:
# CSV 파일 읽기
dong_df = spark.read.csv("hdfs://localhost:9000/data/동polygon.csv", header=True, inferSchema=True)
dong_df

In [ ]:
# 새로운 x, y 컬럼 추가 (WGS84로 변환)
dong_df = dong_df.withColumn("x", utmk_to_wgs84_udf_lon(dong_df["X좌표"], dong_df["Y좌표"]))
dong_df = dong_df.withColumn("y", utmk_to_wgs84_udf_lat(dong_df["X좌표"], dong_df["Y좌표"]))

# geometry 컬럼 좌표 변환
dong_df = dong_df.withColumn("transformed_geometry", transform_polygon_udf(dong_df["geometry"]))

# 결과 출력
dong_df.show(truncate=False)

In [ ]:
dong_df.show()

In [4]:
# CSV 파일 읽기
sang_df = spark.read.csv("hdfs://localhost:9000/data/상권polygon.csv", header=True, inferSchema=True)
sang_df

DataFrame[상권_코드: int, X좌표: double, Y좌표: double, 구코드: int, 구 이름: string, 동코드: int, 동이름: string, geometry: string]

In [9]:
# 새로운 x, y 컬럼 추가 (WGS84로 변환)
sang_df = sang_df.withColumn("x", utmk_to_wgs84_udf_lon(sang_df["X좌표"], sang_df["Y좌표"]))
sang_df = sang_df.withColumn("y", utmk_to_wgs84_udf_lat(sang_df["X좌표"], sang_df["Y좌표"]))

# geometry 컬럼 좌표 변환
sang_df = sang_df.withColumn("transformed_geometry", transform_polygon_udf(sang_df["geometry"]))

# 결과 출력
sang_df.show(truncate=False)

/tmp/ipykernel_98311/222125601.py:3: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/tmp/ipykernel_98311/222125601.py:3: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
 85%|████████▌ | 102/120 [00:00<00:00, 166.17it/s]

+---------+--------+--------+------+-------+--------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

100%|██████████| 120/120 [00:00<00:00, 165.77it/s]


In [ ]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="S11P21D108",
    charset="utf8mb4",  # 문자셋 설정
    collation="utf8mb4_general_ci",  # collation 설정
    autocommit=False,  # Auto-commit을 끄고 트랜잭션 처리
    connection_timeout=28800,  # 타임아웃 증가
)

In [ ]:
# 필요한 컬럼만 선택하여 DataFrame 생성
df_selected = df.select(
    df['구코드'].alias('gu_code'),
    df['구이름'].alias('gu_name'),
    df['x좌표'].alias('x_pos'),
    df['y좌표'].alias('y_pos'),
    df['geometry'].alias('geometry')
)

In [ ]:
# MariaDB에 삽입할 함수 정의
def insert_into_mariadb(row):
    sql = """
    INSERT INTO gu (gu_code, gu_name, x_pos, y_pos, geometry)
    VALUES (%s, %s, %s, %s, ST_GeomFromText(%s))
    """
    cursor.execute(sql, (row['gu_code'], row['gu_name'], row['x_pos'], row['y_pos'], row['geometry']))

In [ ]:
cursor = connection.cursor()

# DataFrame의 각 행을 MariaDB에 삽입
for row in df_selected.collect():
    insert_into_mariadb(row)

# 변경사항 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()